# Coach Slide-Ready Workbook

This notebook is a consumer of Notebook 01 outputs.
It reads `outputs/results.json` and exported tables/figures without recomputing analysis.

## 0. Setup
- Run Notebook 01 first (or `scripts/render_visual_templates.py`).
- This notebook only formats text/tables and aliases figures for deck insertion.

In [ ]:
import shutil

from pathlib import Path
import pandas as pd
from IPython.display import Image, Markdown, display

from browns_tracking.config import default_project_paths
from browns_tracking.pipeline import classify_hsr_exposure
from browns_tracking.presentation import (
    build_slide_1_snapshot_text,
    coach_early_late_table,
    coach_extrema_table,
    coach_peak_distance_table,
    coach_segment_table,
    coach_speed_band_table,
    write_slide_text,
)
from browns_tracking.results_contract import load_results_contract

pd.set_option('display.max_columns', 140)
pd.set_option('display.width', 240)

In [ ]:
paths = default_project_paths()
OUTPUT_DIR = paths.output_dir
FIG_DIR = OUTPUT_DIR / 'figures'
TABLE_DIR = OUTPUT_DIR / 'tables'
TEXT_DIR = OUTPUT_DIR / 'slide_text'
TABLE_DIR.mkdir(parents=True, exist_ok=True)
TEXT_DIR.mkdir(parents=True, exist_ok=True)

contract = load_results_contract(OUTPUT_DIR)
session_summary = contract['session_summary']
qa_summary = contract['qa_summary']
validation_takeaways = contract.get('validation_takeaways', [])
thresholds = contract['thresholds']
hsr_threshold_mph = float(thresholds['hsr_threshold_mph'])
accel_threshold_ms2 = float(thresholds['accel_threshold_ms2'])
decel_threshold_ms2 = float(thresholds['decel_threshold_ms2'])

speed_band_summary = pd.read_csv(TABLE_DIR / 'absolute_speed_band_summary.csv')
distance_table = pd.read_csv(TABLE_DIR / 'peak_distance_windows.csv')
extrema = pd.read_csv(TABLE_DIR / 'session_extrema.csv')
event_counts = pd.read_csv(TABLE_DIR / 'session_event_counts.csv').iloc[0].to_dict()
early_late_summary = pd.read_csv(TABLE_DIR / 'early_vs_late_summary.csv')
coach_phase_summary = pd.read_csv(TABLE_DIR / 'coach_phase_summary.csv')
validation_gates = pd.read_csv(TABLE_DIR / 'validation_gates.csv')
peak_windows = pd.read_csv(OUTPUT_DIR / 'peak_windows.csv')

required_figures = [FIG_DIR / '01_space.png', FIG_DIR / '02_time.png', FIG_DIR / '03_peaks.png']
missing_figures = [str(p) for p in required_figures if not p.exists()]
if missing_figures:
    raise FileNotFoundError(
        'Missing canonical figures. Run Notebook 01 or scripts/render_visual_templates.py first:\n'
        + '\n'.join(missing_figures)
    )

pd.Series({
    'Contract': str(OUTPUT_DIR / 'results.json'),
    'Model': contract['model']['name'],
    'Questions': '; '.join(contract['story_questions']),
}, name='value').to_frame()

In [ ]:
def show_saved_png(path, caption='', width=1250):
    png = Path(path)
    if caption:
        display(Markdown(f'**{caption}**'))
    if not png.exists():
        display(Markdown(f':warning: Missing `{png}`.'))
        return
    display(Image(filename=str(png), width=width))

## Deck Map (Use This 6-Slide Flow)

In [ ]:
deck_outline = pd.DataFrame([
    {
        'slide': 1,
        'title': 'Session Snapshot + Validation Gates',
        'figure_or_table': (
            'outputs/tables/slide_1_data_quality_table.csv; '
            'outputs/tables/slide_1_validation_gates_table.csv'
        ),
        'one_line_takeaway': 'Data quality gates passed at a usable level, so workload outputs are decision-ready.'
    },
    {
        'slide': 2,
        'title': 'Where: Spatial Usage and Role Signature',
        'figure_or_table': 'outputs/figures/01_space.png; outputs/phase_table.csv',
        'one_line_takeaway': 'The session lived in specific field zones and repeated role-consistent movement patterns.'
    },
    {
        'slide': 3,
        'title': 'When: Intensity Timeline and Session Structure',
        'figure_or_table': 'outputs/figures/02_time.png; outputs/tables/slide_3_top_windows_table.csv',
        'one_line_takeaway': 'High-intensity work clustered in identifiable blocks rather than being evenly distributed.'
    },
    {
        'slide': 4,
        'title': 'What: Peak Demands and Repeatable Windows',
        'figure_or_table': (
            'outputs/figures/03_peaks.png; outputs/tables/slide_3_peak_distance_table.csv; '
            'outputs/tables/slide_3_event_counts_table.csv'
        ),
        'one_line_takeaway': 'Top windows define the true worst-case demands to anchor drill and conditioning targets.'
    },
    {
        'slide': 5,
        'title': 'Phase-Level Load Profile',
        'figure_or_table': 'outputs/tables/slide_4_segment_table.csv',
        'one_line_takeaway': 'Merged coach phases show which blocks carried most distance and high-speed stress.'
    },
    {
        'slide': 6,
        'title': 'Early vs Late Readiness Signal',
        'figure_or_table': 'outputs/tables/slide_5_early_late_table.csv',
        'one_line_takeaway': 'Late-session drift quantifies whether second-half output matched training intent.'
    },
])
display(deck_outline)
deck_outline.to_csv(TABLE_DIR / 'final_deck_outline.csv', index=False)

## Coach/Player Visual Report (Auto-Display)

In [ ]:
show_saved_png(
    FIG_DIR / '01_space.png',
    'Slide 2 | Where: Spatial usage and role signature (density + key demand windows)'
)
show_saved_png(
    FIG_DIR / '02_time.png',
    'Slide 3 | When: Intensity over time with phase strip, HSR events, and cumulative distance'
)
show_saved_png(
    FIG_DIR / '03_peaks.png',
    'Slide 4 | What: Peak demand windows and session maxima with event context'
)

## Coach/Player Summary Block

In [ ]:
best_window = peak_windows.iloc[0] if not peak_windows.empty else None
coach_lines = [
    'Coach Lens',
    f"- Session phases: {len(coach_phase_summary)} merged blocks with clear intensity labels.",
    f"- Validation gates passed: {int((validation_gates['status'] == 'PASS').sum())}/{len(validation_gates)}.",
]
player_lines = [
    'Player Lens',
    f"- Peak speed: {float(session_summary['peak_speed_mph']):.2f} mph.",
    f"- HSR distance: {float(event_counts.get('hsr_distance_yd', 0.0)):.1f} yd.",
]
if best_window is not None:
    start = pd.Timestamp(best_window['window_start_utc']).strftime('%H:%M:%S')
    end = pd.Timestamp(best_window['window_end_utc']).strftime('%H:%M:%S')
    coach_lines.append(
        f"- Key drill anchor: best 1-min window {float(best_window['distance_yd']):.1f} yd ({start}-{end} UTC)."
    )
    player_lines.append(
        f"- Best 1-min work rate: {float(best_window['distance_yd']):.1f} yd; target repeat quality, not just volume."
    )
summary_md = '### ' + '\n'.join(coach_lines) + '\n\n### ' + '\n'.join(player_lines)
display(Markdown(summary_md))

## 1. Slide 1: Snapshot + Validation Gates

In [ ]:
slide1_text = build_slide_1_snapshot_text(
    session_summary,
    hsr_threshold_mph=hsr_threshold_mph,
    event_summary=event_counts,
)
print(slide1_text)
write_slide_text(TEXT_DIR / 'slide_1_session_snapshot.txt', slide1_text)

qa_table = pd.DataFrame([qa_summary])[
    [
        'sample_count', 'expected_cadence_s', 'pct_on_expected_cadence',
        'max_gap_s', 'gap_count', 'gap_threshold_s',
        'step_distance_outlier_threshold_yd', 'step_distance_outlier_count',
        'step_distance_outlier_pct',
    ]
].rename(columns={
    'sample_count': 'Sample count',
    'expected_cadence_s': 'Expected cadence (s)',
    'pct_on_expected_cadence': '% at expected cadence',
    'max_gap_s': 'Max gap (s)',
    'gap_count': 'Gap count',
    'gap_threshold_s': 'Gap threshold (s)',
    'step_distance_outlier_threshold_yd': 'Outlier threshold (yd)',
    'step_distance_outlier_count': 'Outlier count',
    'step_distance_outlier_pct': 'Outlier count (%)',
})
for col, digits in [
    ('Expected cadence (s)', 3),
    ('% at expected cadence', 2),
    ('Max gap (s)', 2),
    ('Gap threshold (s)', 2),
    ('Outlier threshold (yd)', 2),
    ('Outlier count (%)', 2),
]:
    qa_table[col] = qa_table[col].round(digits)
display(qa_table)
qa_table.to_csv(TABLE_DIR / 'slide_1_data_quality_table.csv', index=False)

validation_table = validation_gates[['gate', 'status', 'value', 'threshold', 'direction', 'unit', 'notes']].copy()
validation_table.columns = ['Gate', 'Status', 'Value', 'Threshold', 'Direction', 'Unit', 'Notes']
validation_table['Value'] = pd.to_numeric(validation_table['Value'], errors='coerce').round(3)
display(validation_table)
validation_table.to_csv(TABLE_DIR / 'slide_1_validation_gates_table.csv', index=False)

pass_count = int((validation_gates['status'] == 'PASS').sum()) if not validation_gates.empty else 0
qa_text_lines = [
    'Data QA Summary',
    f"- {qa_table['% at expected cadence'].iloc[0]:.1f}% samples at 0.1s cadence.",
    f"- Max gap: {qa_table['Max gap (s)'].iloc[0]:.2f}s; gaps flagged above {qa_table['Gap threshold (s)'].iloc[0]:.2f}s.",
    f"- Outlier threshold: {qa_table['Outlier threshold (yd)'].iloc[0]:.2f} yd; flagged {int(qa_table['Outlier count'].iloc[0])} samples ({qa_table['Outlier count (%)'].iloc[0]:.2f}%).",
    f"- Validation gates passed: {pass_count}/{len(validation_gates)}.",
]
qa_text_lines.extend([f"- {line}" for line in validation_takeaways[:3]])
qa_text = '\n'.join(qa_text_lines)
print(qa_text)
write_slide_text(TEXT_DIR / 'slide_1_data_quality_takeaways.txt', qa_text)

## 2. Slide 2: Speed Zones

In [ ]:
slide2_table = coach_speed_band_table(speed_band_summary)
display(slide2_table)
slide2_table.to_csv(TABLE_DIR / 'slide_2_speed_zone_table.csv', index=False)

top_zone = slide2_table.sort_values('Distance (yd)', ascending=False).iloc[0]
hsr_label = classify_hsr_exposure(
    total_distance_yd=float(session_summary['distance_yd_from_speed']),
    hsr_distance_yd=float(event_counts.get('hsr_distance_yd', 0.0)),
)
slide2_text = (
    'Speed Zone Takeaways\n'
    f"- Largest distance accumulation: {top_zone['Zone']} ({top_zone['Distance (%)']:.1f}% of total distance).\n"
    f"- HSR exposure classification: {hsr_label}.\n"
    '- Action: adjust high-speed volume by phase, not by session average only.'
)
print(slide2_text)
write_slide_text(TEXT_DIR / 'slide_2_speed_zone_takeaways.txt', slide2_text)

## 3. Slide 3: Peak Demands

In [ ]:
slide3_distance = coach_peak_distance_table(distance_table)
slide3_extrema = coach_extrema_table(extrema)
slide3_events = pd.DataFrame([
    {
        'HSR events (>=1s)': int(event_counts.get('hsr_event_count', 0)),
        'Sprint events (>=1s)': int(event_counts.get('sprint_event_count', 0)),
        'Accel events': int(event_counts.get('accel_event_count', 0)),
        'Decel events': int(event_counts.get('decel_event_count', 0)),
        'HSR distance (yd)': float(event_counts.get('hsr_distance_yd', 0.0)),
        'Sprint distance (yd)': float(event_counts.get('sprint_distance_yd', 0.0)),
    }
])
slide3_events['HSR distance (yd)'] = slide3_events['HSR distance (yd)'].round(1)
slide3_events['Sprint distance (yd)'] = slide3_events['Sprint distance (yd)'].round(1)

slide3_top_windows = peak_windows[
    ['window_rank', 'window_start_utc', 'window_end_utc', 'distance_yd', 'dominant_phase']
].copy().rename(columns={
    'window_rank': 'Window',
    'window_start_utc': 'Start (UTC)',
    'window_end_utc': 'End (UTC)',
    'distance_yd': 'Distance in window (yd)',
    'dominant_phase': 'Dominant phase',
})
slide3_top_windows['Start (UTC)'] = pd.to_datetime(
    slide3_top_windows['Start (UTC)'],
    utc=True,
    format='mixed',
).dt.strftime('%H:%M:%S')
slide3_top_windows['End (UTC)'] = pd.to_datetime(
    slide3_top_windows['End (UTC)'],
    utc=True,
    format='mixed',
).dt.strftime('%H:%M:%S')
slide3_top_windows['Distance in window (yd)'] = slide3_top_windows['Distance in window (yd)'].round(1)

display(slide3_distance)
display(slide3_extrema)
display(slide3_events)
display(slide3_top_windows)

slide3_distance.to_csv(TABLE_DIR / 'slide_3_peak_distance_table.csv', index=False)
slide3_extrema.to_csv(TABLE_DIR / 'slide_3_extrema_table.csv', index=False)
slide3_events.to_csv(TABLE_DIR / 'slide_3_event_counts_table.csv', index=False)
slide3_top_windows.to_csv(TABLE_DIR / 'slide_3_top_windows_table.csv', index=False)

if slide3_top_windows.empty:
    slide3_text = 'Peak Demand Takeaways\n- Not enough samples to derive stable top-demand windows.'
else:
    best = peak_windows.iloc[0]
    best_start = pd.Timestamp(best['window_start_utc']).strftime('%H:%M:%S')
    best_end = pd.Timestamp(best['window_end_utc']).strftime('%H:%M:%S')
    slide3_text = (
        'Peak Demand Takeaways\n'
        f"- Best 1-min demand: {float(best['distance_yd']):.1f} yd from {best_start} to {best_end} UTC.\n"
        f"- Context: HSR/Sprint {int(best['hsr_event_count'])}/{int(best['sprint_event_count'])}; Acc/Dec {int(best['accel_event_count'])}/{int(best['decel_event_count'])}.\n"
        f"- Dominant phase: {best['dominant_phase'] or 'N/A'}.\n"
        '- Action: use this window to set drill-level peak-demand targets.'
    )
print(slide3_text)
write_slide_text(TEXT_DIR / 'slide_3_peak_takeaways.txt', slide3_text)

## 4. Slide 4: Session Phases

In [ ]:
slide4_table = coach_segment_table(coach_phase_summary, top_n=8)
display(slide4_table)
slide4_table.to_csv(TABLE_DIR / 'slide_4_segment_table.csv', index=False)

top_phase = coach_phase_summary.sort_values('distance_yd', ascending=False).iloc[0]
high_phase_count = int((coach_phase_summary['intensity_level'] == 'High').sum())
slide4_text = (
    'Session Phase Takeaways\n'
    f"- Session merged into {len(coach_phase_summary)} coach-readable phases.\n"
    f"- Highest volume phase: {top_phase['coach_phase_label']} ({top_phase['distance_yd']:.1f} yd across {top_phase['duration_s'] / 60.0:.1f} min).\n"
    f"- High-intensity phases identified: {high_phase_count}.\n"
    '- Action: use phase boundaries as planning blocks for next session design.'
)
print(slide4_text)
write_slide_text(TEXT_DIR / 'slide_4_segment_takeaways.txt', slide4_text)

## 5. Slide 5: Early vs Late

In [ ]:
slide5_table = coach_early_late_table(early_late_summary)
display(slide5_table)
slide5_table.to_csv(TABLE_DIR / 'slide_5_early_late_table.csv', index=False)

if len(slide5_table) == 2:
    late = slide5_table.loc[slide5_table['Period'] == 'Late Half'].iloc[0]
    slide5_text = (
        'Early vs Late Takeaways\n'
        f"- Late-half distance vs early-half: {late['Distance vs early (%)']:+.1f}%.\n"
        f"- Late-half HSR/Sprint events: {int(late['HSR events'])}/{int(late['Sprint events'])}.\n"
        f"- Late-half accel/decel events: {int(late['Accel events'])}/{int(late['Decel events'])}.\n"
        '- Action: adjust second-half progression if high-speed or braking load drifts from intent.'
    )
else:
    slide5_text = 'Early vs Late Takeaways\n- Insufficient data for split-half comparison.'
print(slide5_text)
write_slide_text(TEXT_DIR / 'slide_5_early_late_takeaways.txt', slide5_text)

## 6. Figure Aliases for Slide Deck

In [ ]:
figure_aliases = {
    '01_space.png': 'coach_slide_movement_map.png',
    '02_time.png': 'coach_slide_intensity_timeline.png',
    '03_peaks.png': 'coach_slide_peak_demand_summary.png',
}
for src, dst in figure_aliases.items():
    source = FIG_DIR / src
    if not source.exists():
        raise FileNotFoundError(f'Missing {source}. Run Notebook 01 first.')
    shutil.copyfile(source, FIG_DIR / dst)

('Saved aliases', FIG_DIR / 'coach_slide_movement_map.png', FIG_DIR / 'coach_slide_intensity_timeline.png', FIG_DIR / 'coach_slide_peak_demand_summary.png')